# Incroduction
- Chp1에선 hihg-level의 pipline API를 사용하여 여러 task에 대한 transformer model을 사용해봤다.
- pipeline api가 편하더라도, 이것에 대한 이해를 바탕으로 해야 더 유연하게 문제를 해결할 수 있다.
- 이번 챕터에선 이 부분에 대해서 알아보고자 한다.

# Behind the pipeline
Chp 1을 상기해보면, 특정 task에 해당하는 pipeline을 불러오고, 여기에 inputs을 넣어서 결과를 출력했다.

이 과정을 간략히 본다면 다음과 같다.
- Raw Text input => Tokenizer -> input ids => Model -> logits=> Post-processing -> predictions
아래와 같이 결과가 나오고, Chp2에서는 이 과정들을 살펴보자

In [1]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
classifier([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

## Preprocessing with a tokenizer
- Pipeline의 첫 step은 raw input을 token으로 변경 후 정수로 변환해주는 단계이며, 이 과정에서 tokenizer를 사용
- AutoTokenizer 클래스의 from_pretrained를 사용


In [3]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

- raw input이 tokenizer를 통과하면 모델에 넣을 수 있는 형태의 input으로 변경됨
- 이제 입력 input IDs를 Tensor 형태로 변환이 필요

In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True,return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

padding과 truncation은 이후에 설명하고,   
우선 Pytorch tensor로 변환하기 위해 return_tensors="pt"로 지정하면 위와 같이 출력된다.   
input_ids는 각 seq에 대해 토큰화 후 정수인코딩을 한 결과이다.   
attention_mask는 이후 챕터에서 설명!!

## Going through the model
사용할 모델은 tokenizer와 동일한 방법으로 호출이 가능하다

In [5]:
from transformers import AutoModel
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


이 모델 구조는  base Transformer module만 포함되어 있어, inputs이 주어지면 hidden states의 outputs이 출력된다.   
hidden states 자체도 유용하지만, 일반적으로 이 outputs은 다른 part의 inputs으로 사용된다.   
Chp1을 상기해보면, 여러 task가 있지만, 동일한 architecture의 model을 사용하고 마지막 head만 다르다.

## A high-dimensional vector?
transformer의 output vector는 다음과 같은 차원을 갖는다.
- batch size : sequence 개수
- sequence length : sequence 길이
- hidden size : vector dimension

여기서 high-dimensional vector라고 불리는 이유는, vector의 마지막 rank의 차원이 매우 크기 때문이다. (128,256,512,768,3072,or more...)   

model에 input을 넣는 방법은 다음과 같다

In [18]:
print("inputs : ", inputs)
outputs = model(**inputs)
print("*"*100)
print(outputs.keys())
print("outputs_size : ", outputs.last_hidden_state.shape)
print("outputs : ", outputs.last_hidden_state)


inputs :  {'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}
****************************************************************************************************
odict_keys(['last_hidden_state'])
outputs_size :  torch.Size([2, 16, 768])
outputs :  tensor([[[-0.1798,  0.2333,  0.6321,  ..., -0.3017,  0.5008,  0.1481],
         [ 0.2758,  0.6497,  0.3200,  ..., -0.0760,  0.5136,  0.1329],
         [ 0.9046,  0.0985,  0.2950,  ...,  0.3352, -0.1407, -0.6464],
         ...,
         [ 0.1466,  0.5661,  0.3235,  ..., -0.3376,  0.5100, -0.0561],
         [ 0.7500,  0.0487,  0.1738,  ...,  0.4684,  0.0030, -0.6084],
         [ 0.0519,  

## Model heads: Making sense out of numbers

model head는 hidden state vectors를 input으로 하여 다른 차원으로 투영하고 결과를 출력하며, 다음과 같이 진행된다.
![](https://huggingface.co/course/static/chapter2/transformer_and_head.png)

outputs이 특정 task를 위한 head의 input으로 사용된다.   
transformers에는 다양한 architectures가 있으며, 각 architecture는 특정 작업에 focus되어 설계되었다.   
다음은 transformers에 있는 architecture를 불러오기 위한 일부 방법이다.
- *Model (retrieve the hidden states)
- *ForCausalLM
- *ForMaskedLM
- *ForMultipleChoice
- *ForQuestionAnswering
- *ForSequenceClassification
- *ForTokenClassification
- and others 🤗

한 예로, sequence classification head를 위한 model architecture를 불러오려면, automodel이 아니라 ForSequenceClassification이다.


In [20]:
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [23]:
outputs = model(**inputs)
print("inputs: ", inputs)
print("outputs: ", outputs)

inputs:  {'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}
outputs:  SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


해당 결과는 logits으로 softmax를 통과하여 감성분류 결과를 확인해야 한다.

In [26]:
import torch
pred = torch.nn.functional.softmax(outputs.logits,dim=-1)
pred

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)

이제 해당 값을 기준으로 label을 지정해야 하며 이때, model의 method중 id2label를 보면 각 label을 확인 가능하다

In [28]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

# Models
- model을 만들고 사용하는 방법에 대해 알아보기
- 일반적으로 checkpoint로부터 모델을 인스턴스화 할 때는 automodel을 사용
- AutoModel은 입력한 checkpoint를 기반으로 적한한 모델 아키텍처를 자동으로 추축한 다음 불러오므로 매우 사용하기 편리함
- 단, 사용할 모델 유형을 알고 있는 경우 직접 사용 가능
- bert model로 이 기능이 어떻게 동작하는지 확인해보자

## Creating a Transformer
### Different loading methods
우선 BERT model을 만들기에 앞서 우선 configuration을 load해야 한다.


In [36]:
from transformers import BertConfig, BertModel

# Buliding the config
config = BertConfig()
print("config for BERT", "\n",config)

# Building the model from the config
model = BertModel(config)
print("*"*50)
print(model)

config for BERT 
 BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

**************************************************
BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
   

model config를 불러왔지만, weight와 bias가 랜덤초기화가 된 상태다. 따라서 해당 상태로 그냥 사용할 순 없고 학습을 시켜야 한다. 다만 학습에는 많은 cost와 data가 필요하므로, 불필요한 중복학습을 방지하기 위해선 이미 학습되어 공유된 모델을 재사용하면 된다.

이 때 from_pretrained method를 활용하면 된다.

In [37]:
from transformers import BertModel
model = BertModel.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


앞서 실습한 것과 같이, BertModel로 AutoModel 대체 가능하다.  

지금부터 진행할 실습에서는 checkpoint-agnostic한 코드를 사용 할 예정이므로, 한 체크포인트에서 코드가 잘 동작하면 다른 체크포인트에서도 완벽히 동작할 것이다.   

이 코드에서는 BertConfig를 사용하지 않고, Bert-base-case 식별자를 통해 모델을 로드했다. 이는 Bert 작성자가 직접 교육한 모델의 체크포인트로, 모델에 대한 자세한 것은 modelcard를 참고하면 된다. 

불러온 모델은 체크포인트의 모든 가중치로 초기화되며, downstream task를 위해 finetune만 수행하면 되므로, 빠르게 학습하여 좋은 결과를 얻을 수 있다.

가중치가 캐시 폴더에 다운로드 및 캐시화되어 향후 다시 다운할 필요는 없고, 기본적으로 ~/chche/huggingface/transformers 위치에서 진행된다.

HF_HOME 환경변수를 설정하여 customizing이 가능하다.

### Saving methods
모델 save는 save_pretrained moethod를 사용하면 된다.

In [38]:
model.save_pretrained("directory_on_my_computer")

## Using a Transformer model for inference
model은 tokenizer에서  생성한 inputs에 대해서만 처리가 가능하다. 토크나이저를 다루기 이전에, 모델이 어떤 inputs을 받는지 알아보자

In [45]:
sequences = [
  "Hello!",
  "Cool.",
  "Nice!"
]

encoded_sequences = [
  [ 101, 7592,  999,  102],
  [ 101, 4658, 1012,  102],
  [ 101, 3835,  999,  102]
]
encoded_sequences

[[101, 7592, 999, 102], [101, 4658, 1012, 102], [101, 3835, 999, 102]]

tokenizers는 input을 vocab의 idex로 mapping을 하는데, 상기 과정에서 sequences가 encoded_sequenced로 정수인코딩이 되었다.

torch의 inputs은 tensor이므로 변환을 해준다.

In [44]:
import torch
model_inputs = torch.tensor(encoded_sequences)
model_inputs

tensor([[ 101, 7592,  999,  102],
        [ 101, 4658, 1012,  102],
        [ 101, 3835,  999,  102]])

### Using the tensors as inputs to the model
이렇게 만든 tensor를 model의 input으로 넣어준다.

In [57]:
output = model(model_inputs)

# Tokenizers
- NLP 파이프라인의 핵심 구성 요소 중 하나
- 텍스트를 모델이 처리가능한 형태로 변환(str-> int)
- 몇 가지 토큰화 알고리즘을 알아보고, 몇 가지 질문에 대해 대답을 해보자

## Word-based
- spacebar 혹은 spacebar + punctuation 기준으로 나눌 수 있음
- 매우 쉬운 방법으로 종종 적절한 결과를 산출
- 단, 모든 단어에 대한 vocabulary를 만드려면 매우 큰 courpus가 필요
  - computer는 run과 runs, running, ran 등을 다 다른 단어로 판단
- 하지만, 그럼에도 없는 단어는 <UNK>으로 사용하는 unknown token을 사용하여 mapping시킨다.
- 또 다른 방법으로는 더 작은 단위로 token을 쪼개는 것이다.

## Character-based
- character level로 토큰화를 진행하면 다음과 같은 장단점이 있음
  - pros : small vocabulary, unk token을 최소화 가능
  - cons : 
    - 언어에 따라 하나의 character가 담고있는 의미의 크기가 다를 수 있음
    - 매우 긴 sequence를 token화하면 너무 커짐
  - 따라서 character level과 word level의 중간인 subword level이 있음

## Subward tokenization
- principle : 자주 사용되는 단어는 분해 X, 희귀한 단어는 하위 단어로 분해
  - e.g.) annoyingly => annoying, ly
  - annoying과 ly는 기존보다 이 상태에서 더 자주 등장하므로 분해
- 해당 방식을 활용하면 unk token 사용을 최소화 가능
- subword에는 다양한 알고리즘이 존재
  - Byte-level BPE, as used in GPT-2
    - 빈도수 기반
  - WordPiece, as used in BERT
    - BPE와 같지만 likelihood 기반
  - SentencePiece or Unigram, as used in several multilingual models
    - BPE + unigram-LM 기반


## Loading and saving
- tokenizer의 save & load는 model과 동일
  - .from_pretrained(체크포인트), save_pretrained(체크포인트)
- 모델이 가중치를 저장하는 것과 같이, tokenizer는 vocab을 저장
- tokenizer도 AutoTokenizer도 있으며, BertTokenizer를 가져오는 것과 동일하게 가져올 수 있음

In [58]:
from transformers import AutoTokenizer, BertTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer_ = BertTokenizer.from_pretrained("bert-base-cased")

In [71]:
print("AutoTokenizer: ",tokenizer)
print("*"*50)
print("BertTokenizer: ",tokenizer_)

AutoTokenizer:  PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
**************************************************
BertTokenizer:  PreTrainedTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [76]:
for k,v in tokenizer("Using a Transformer network is simple").items():
    print(f"{k} : {v}")

input_ids : [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# save tokenizer 
tokenizer.save_pretrained("directory_on_my_computer")

- token_type_ids는 chp3에서 다룰 예정   

일단은 attention mask와 inputs에 대해 공부해보자

## Encoding
- 인코딩 : 텍스트를 숫자로 변환하는 과정
- two-step : 1) tokenization 2) 정수로 변환
- tokenizer마다 저마다의 규칙이 있으므로, 모델의 이름을 사용하여 tokenizer를 저장하고, 불러올 땐 tokenizer도 함께 불러와서 사용해야 함.
- vocabulary 역시도 동일한 것을 사용해야 하며, 이 두 가지를 해주는 것이 from_pretrained() 함수

### Tokenization : seq를 token화 하는 과정

In [78]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)
print("bert tokenizer 사용 결과","\n",tokens)

bert tokenizer 사용 결과 
 ['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


### From tokens to input IDs

In [82]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


## Decoding

In [84]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple


decode는 generate task에서 매우 우용하게 사용!

# Handling multiple sequences
- 기존 section에서 간단한 예제를 살펴봤지만 다음과 같이 의문이 생길 수 있음
  - 여러 문장을 다루는 방법
  - 길이가 다른 문장들을 다루는 방법
  - 정수인코딩만이 모델을 잘 작동하기 위한 방법인지
  - 매우 긴 문장에 대해 다루는 방법
- 이번 section에서 해당 의문들에 대해 속시원하게 풀고 갈 예정

## Models expect a batch of inputs
우선 기존 section에서와 같이, sequence를 숫자로 mapping한 다음 tensor로 변환하는 과정을 진행

In [91]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [103]:
sequence = "I've been waiting for a HuggingFace course my whole life."
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])
output = model(input_ids)

print("Input_ids")
print(input_ids)
print("logits")
print(output.logits)


Input_ids
tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
logits
tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward>)


여러 문장을 처리하는 것은 batching이라 하며 문장을 하나로 묶어서 인풋으로 넣어줌!

In [111]:
batch_ids = [ids,ids]
input_ids = torch.tensor(batch_ids)
output = model(input_ids)

print("Input_ids")
print(input_ids)
print("logits")
print(output.logits)

Input_ids
tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
logits
tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward>)


단 tensor는 직사각형 형태여야 하므로, 길이가 다르면 tensor로 변환이 불가하다

## Padding
길이가 다른 sequence에 대해 padding을 해주면 해결이 가능하다

In [115]:
#two-setences
batched_ids = [
  [200, 200, 200],
  [200, 200],
]

#padding id
padding_id = 100
batched_ids[1].append(padding_id)

pad token은 tokenizer.pad_token_id로 찾을 수 있음

In [117]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [[200, 200, 200], [200, 200, tokenizer.pad_token_id]]

print(tokenizer.pad_token_id)
print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

0
tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)
tensor([[ 1.5694, -1.3895],
        [ 1.3373, -1.2163]], grad_fn=<AddmmBackward>)


우리가 생각하기에, seq1 + seq2 == batched_ids여야 하지만 실제로는 seq2가 다른 결과인 것을 볼 수 있다.

이는 padding token이 영향을 준 것이다. 따라서 attention mask를 사용하여 패딩 토큰을 무시하면 된다.

## Attention masks
입력 텐서에 대해 0,1로 구분하는데 여기서 1은 연산을 진행하는 값들이고, 0은 무시하는 값이다

In [142]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id]
]

attention_mask = [
  [1, 1, 1],
  [1, 1, 0]
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)


In [151]:
#try out
raw_inputs = ["I’ve been waiting for a HuggingFace course my whole life.",
"I hate this so much!"]

#tokenization
raw1 = tokenizer([raw_inputs[0]], return_tensors="pt")
raw2 = tokenizer([raw_inputs[1]], return_tensors="pt")

tokenized_inputs = tokenizer(inputs,padding=True, return_tensors="pt")
tokenized_inputs
raw1
print("raw1")
print(model(**raw1))
print("raw2")
print(model(**raw2))
print("all")
print(model(**tokenized_inputs))

raw1
SequenceClassifierOutput(loss=None, logits=tensor([[-1.5979,  1.6390]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
raw2
SequenceClassifierOutput(loss=None, logits=tensor([[ 4.1692, -3.3464]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
all
SequenceClassifierOutput(loss=None, logits=tensor([[-1.5979,  1.6390],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


## Longer sequences
- BERT 기준 sequence는 512에서 1024개의 token 개수로 제한되어 있음
- 더 긴 문장을 해결하는 방법
  - 1) 더 긴 문장을 다룰 수 있는 모델 사용
    - [Longformer](https://huggingface.co/transformers/model_doc/longformer.html), [LED](https://huggingface.co/transformers/model_doc/led.html)
  - 2) truncation
    - sequence = sequence[:max_sequence_length]


# Putting it all together
- 이전 과정들에서 토크나이저 작동 방법, 토큰화, input_ids로 변환, 패딩, truncation등을 살펴봤는데
- transformers API에서는 이러한 기능들을 모두 제공

In [153]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
model_inputs

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

model input 변수로는 model 작동하기 위해 다양한 attribute가 있는데, distillbert의 경우는 attention mask도 필요한 경우가 있다.

상기 방법에서 multi-sequences에 대햇도 API변경 없이 그대로 사용할 수 있다

In [155]:
sequences = [
  "I've been waiting for a HuggingFace course my whole life.",
  "So have I!"
]

model_inputs = tokenizer(sequences)
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

#padding을 해주는 방법은 다양하다.

In [162]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")
# longest : default
print(model_inputs)
print("*"*50)
# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")
print(model_inputs)
print("*"*50)
# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)
print(model_inputs)
print("*"*50)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
**************************************************
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

긴 문장에 대해선 다음과 같이 자를 수 있다.

In [165]:
sequences = [
  "I've been waiting for a HuggingFace course my whole life.",
  "So have I!"
]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)
print(model_inputs)
print("*"*50)
# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)
print(model_inputs)
print("*"*50)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}
**************************************************
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}
**************************************************


tokenzer의 return 형태로는 세 가지가 가능하다
- numpy arrays : "np"
- PyTorch tensors : "pt"
- TensorFlow tensors : "tf"

In [172]:
sequences = [
  "I've been waiting for a HuggingFace course my whole life.",
  "So have I!"
]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")
print(model_inputs)
print("*"*50)
# Returns TensorFlow tensors
# model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")
# print(model_inputs)
# print("*"*50)
# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")
print(model_inputs)
print("*"*50)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
**************************************************
{'input_ids': array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
**************************************************


## Special tokens
- 토크나이저 출력 결과를 보면 기존과 다른 것을 알 수 있음
- 시작 부분에 [CLS], 끝에는 [SEP] 스페셜 토큰이 추가됨
- 이는 모델의 전처리 방법을 따르기 때문
- 모델에 따라 스페셜 토큰이 다름

In [176]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids,"\n")

print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012] 

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


## Wrapping up: From tokenizer to model
이제 토크나이저로 전처리 시 모든 간계를 알았기 한 번 해보자고오오오

In [185]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences = [
  "I've been waiting for a HuggingFace course my whole life.",
  "So have I!"
]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)


In [207]:
import numpy as np
output_ = torch.nn.functional.softmax(output.logits,dim=-1)
np.round(output_.detach().numpy())
# output

array([[0., 1.],
       [0., 1.]], dtype=float32)